# Customer Revenue Prediction

The 80/20 rule has proven true for many businesses–only a small percentage of customers produce most of the revenue. As such, marketing teams are challenged to make appropriate investments in promotional strategies.
This notebook will analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset from Kaggle competition ['Google Analytics Customer Revenue Prediction'](https://www.kaggle.com/c/ga-customer-revenue-prediction) to predict revenue per customer. 

### About the dataset:

Each row in the dataset is one visit to the store. Targeted outputs are prediction of the natural log of the sum of all transactions per user.

The data fields in the given files are

* fullVisitorId- A unique identifier for each user of the Google Merchandise Store.
* channelGrouping - The channel via which the user came to the Store.
* date - The date on which the user visited the Store.
* device - The specifications for the device used to access the Store.
* geoNetwork - This section contains information about the geography of the user.
* sessionId - A unique identifier for this visit to the store.
* socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
* totals - This section contains aggregate values across the session.
* trafficSource - This section contains information about the Traffic Source from which the session originated.
* visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
* visitNumber - The session number for this user. If this is the first session, then this is set to 1.
* visitStartTime - The timestamp (expressed as POSIX time).

Note: some of the fields are in json format, the dataset size is very large. The train and test datasets used in this analysis is a cvs file with json flattened from Kaggle [json flattened csv data](https://www.kaggle.com/colinpearse/ga-analytics-with-json-columns)(thanks to Collin!).

In [1]:
import os
import gc   # garbage collector
import numpy as np
import pandas as pd
import datetime

import warnings
warnings.filterwarnings("ignore")

## Load Data and Preprocessing

In [2]:
train_df = pd.read_csv ('./datasets/train.csv')
train_df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,organic,NaN,google


In [3]:
test_df = pd.read_csv('./datasets/test.csv')
test_df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20171016,6167871330617112363,6167871330617112363_1508151024,Not Socially Engaged,1508151024,2,1508151024,Chrome,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,organic,NaN,google
1,Organic Search,20171016,0643697640977915618,0643697640977915618_1508175522,Not Socially Engaged,1508175522,1,1508175522,Chrome,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
2,Organic Search,20171016,6059383810968229466,6059383810968229466_1508143220,Not Socially Engaged,1508143220,1,1508143220,Chrome,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
3,Organic Search,20171016,2376720078563423631,2376720078563423631_1508193530,Not Socially Engaged,1508193530,1,1508193530,Safari,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
4,Organic Search,20171016,2314544520795440038,2314544520795440038_1508217442,Not Socially Engaged,1508217442,1,1508217442,Safari,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google


In [4]:
dif_col = [col for col in train_df.columns if col not in test_df.columns]
print('There are {} columns in train data and not in test data : \n{}'.format(len(dif_col),dif_col))

There are 2 columns in train data and not in test data : 
['totals.transactionRevenue', 'trafficSource.campaignCode']


In [5]:
cols_to_drop = [col for col in train_df.columns if train_df[col].nunique(dropna = False) == 1]
print('{} columns containing no information or only 1 data,will be dropped for further analysis, including: \n{}'.format(len(cols_to_drop), cols_to_drop))

19 columns containing no information or only 1 data,will be dropped for further analysis, including: 
['socialEngagementType', 'device.browserSize', 'device.browserVersion', 'device.flashVersion', 'device.language', 'device.mobileDeviceBranding', 'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.mobileDeviceModel', 'device.mobileInputSelector', 'device.operatingSystemVersion', 'device.screenColors', 'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits', 'trafficSource.adwordsClickInfo.criteriaParameters']


In [6]:
# drop the empty columns and the one column that's not included in test data

train_df.drop(cols_to_drop + ['trafficSource.campaignCode'], axis=1, inplace=True)
test_df.drop(cols_to_drop, axis=1, inplace=True)
train_df.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,desktop,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,UC Browser,desktop,False,...,NaN,NaN,NaN,NaN,NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,1472822600,2,1472822600,Chrome,mobile,True,...,NaN,NaN,NaN,NaN,NaN,True,NaN,organic,NaN,google


In [7]:
gc.collect()

def NaN_stat(df):
    missing_vl = pd.DataFrame(df.isnull().sum(), columns = ['Count'])
    missing_vl['Percent(%)'] = round(missing_vl['Count']/len(df) * 100, 2)
    missing_vl = missing_vl.sort_values('Count')
    return missing_vl
train_missing = NaN_stat(train_df)
train_missing

,Count,Percent(%)
channelGrouping,0,0.00
date,0,0.00
fullVisitorId,0,0.00
sessionId,0,0.00
visitId,0,0.00
visitNumber,0,0.00
visitStartTime,0,0.00
device.deviceCategory,0,0.00
device.isMobile,0,0.00
totals.hits,0,0.00


In [8]:
test_missing = NaN_stat(test_df)
missingdata_df = train_missing.merge(test_missing, left_index = True, right_index = True)
missingdata_df

,Count_x,Percent(%)_x,Count_y,Percent(%)_y
channelGrouping,0,0.00,0,0.00
date,0,0.00,0,0.00
fullVisitorId,0,0.00,0,0.00
sessionId,0,0.00,0,0.00
visitId,0,0.00,0,0.00
visitNumber,0,0.00,0,0.00
visitStartTime,0,0.00,0,0.00
device.deviceCategory,0,0.00,0,0.00
device.isMobile,0,0.00,0,0.00
totals.hits,0,0.00,0,0.00


In [9]:
feature_drop = [i for i in missingdata_df.index if missingdata_df.loc[i,'Percent(%)_x']> 40 and missingdata_df.loc[i,'Percent(%)_y']> 40]
print('{} columns containing ~ 50% missing values in both train and test data, will be dropped for further analysis, including: \n{}'.format(len(feature_drop), feature_drop))

14 columns containing ~ 50% missing values in both train and test data, will be dropped for further analysis, including: 
['totals.bounces', 'geoNetwork.region', 'geoNetwork.city', 'trafficSource.referralPath', 'trafficSource.isTrueDirect', 'geoNetwork.metro', 'trafficSource.campaign', 'trafficSource.keyword', 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.adwordsClickInfo.page', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.slot', 'trafficSource.adContent']


In [10]:
train_df.drop(columns = feature_drop, inplace=True)
test_df.drop(columns = feature_drop, inplace=True)
train_df.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,...,geoNetwork.continent,geoNetwork.country,geoNetwork.networkDomain,geoNetwork.subContinent,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.medium,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,desktop,False,...,Asia,Turkey,ttnet.com.tr,Western Asia,1,1.0,1.0,NaN,organic,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,desktop,False,...,Oceania,Australia,dodo.net.au,Australasia,1,1.0,1.0,NaN,organic,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,Chrome,desktop,False,...,Europe,Spain,unknown.unknown,Southern Europe,1,1.0,1.0,NaN,organic,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,UC Browser,desktop,False,...,Asia,Indonesia,unknown.unknown,Southeast Asia,1,1.0,1.0,NaN,organic,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,1472822600,2,1472822600,Chrome,mobile,True,...,Europe,United Kingdom,unknown.unknown,Northern Europe,1,NaN,1.0,NaN,organic,google


In [11]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 21 columns):
channelGrouping              903653 non-null object
date                         903653 non-null int64
fullVisitorId                903653 non-null object
sessionId                    903653 non-null object
visitId                      903653 non-null int64
visitNumber                  903653 non-null int64
visitStartTime               903653 non-null int64
device.browser               903645 non-null object
device.deviceCategory        903653 non-null object
device.isMobile              903653 non-null bool
device.operatingSystem       898958 non-null object
geoNetwork.continent         902185 non-null object
geoNetwork.country           902185 non-null object
geoNetwork.networkDomain     658772 non-null object
geoNetwork.subContinent      902185 non-null object
totals.hits                  903653 non-null int64
totals.newVisits             703060 non-null float64
totals.page

In [12]:
# process date into datetime format
train_df['date'] = train_df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
test_df['date'] = test_df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))

In [13]:
# fill empty revenue cell with 0
train_df['totals.transactionRevenue'] = train_df['totals.transactionRevenue'].fillna(value=0)

In [14]:
print('Train data sample size : {} \nTest data sample size : {} \nNumber of features to further explore: {} '
      .format(len(train_df), len(test_df), len(test_df.columns)))

print('Within these features: \n   visitId has the same value as visitStartTime \n   sessionId has the combined information of fullVisitorId and visitId')

Train data sample size : 903653 
Test data sample size : 804684 
Number of features to further explore: 20 
Within these features: 
   visitId has the same value as visitStartTime 
   sessionId has the combined information of fullVisitorId and visitId


In [15]:
# save processed train and test data for further analysis
output_dir = os.path.join('./', 'Output')
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
train_df.to_csv(os.path.join(output_dir, 'train_p.csv'))
test_df.to_csv(os.path.join(output_dir, 'test_p.csv'))